In [0]:
from databricks.labs.dqx.profiler.profiler import DQProfiler
from databricks.labs.dqx.profiler.generator import DQGenerator
from databricks.labs.dqx.profiler.dlt_generator import DQDltGenerator
from databricks.labs.dqx.engine import DQEngine
from databricks.sdk import WorkspaceClient
from databricks.labs.dqx.config import OutputConfig

dq_engine = DQEngine(WorkspaceClient())

checks = dq_engine.load_checks_from_workspace_file("/Shared/App1/checks.yml")

input_df = spark.read.table("nprod.tlc.yellow_trip")

valid_df, quarantine_df = dq_engine.apply_checks_by_metadata_and_split(input_df, checks)

dq_engine.save_results_in_table(
  output_df=valid_df, 
  quarantine_df=quarantine_df, 
  output_config=OutputConfig("nprod.tlc.yellow_trip_valid", mode="overwrite")
)